In [2]:

import numpy as np
from dash import Dash, html, dcc, Input, Output, State, clientside_callback

# Example from https://stackoverflow.com/a/63681810/2428887
resolution = 1000
t = np.linspace(0, np.pi * 2, resolution)
x, y = np.cos(t), np.sin(t)  # example data (a circle)
figure = dict(
    data=[{"x": [], "y": []}],
    layout=dict(xaxis=dict(range=[-1, 1]), yaxis=dict(range=[-1, 1])),
)
app = Dash(__name__, update_title=None)  # remove "Updating..." from title
app.layout = html.Div(
    [
        html.H4("Smooth updates using clientside callbacks"),
        dcc.Graph(id="graph", figure=dict(figure)),
        dcc.Interval(id="interval", interval=25, max_intervals=200),
        dcc.Store(id="offset", data=0),
        dcc.Store(id="store", data=dict(x=x, y=y, resolution=resolution)),
    ]
)
clientside_callback(
    """
    function (n_intervals, data, offset) {
        offset = offset % data.x.length;
        const end = Math.min((offset + 10), data.x.length);
        return [[{x: [data.x.slice(offset, end)], y: [data.y.slice(offset, end)]}, [0], 500], end]
    }
    """,
    [
        Output("graph", "extendData"),
        Output("offset", "data"),
    ],
    [Input("interval", "n_intervals")],
    [State("store", "data"), State("offset", "data")],
)

if __name__ == "__main__":
    app.run_server()

In [3]:
import numpy as np
from dash import Dash, html, dcc, Input, Output, clientside_callback
from scipy.integrate import solve_ivp

# Lorenz equations
def lorenz(t, state, sigma=10, rho=28, beta=8/3):
    x, y, z = state
    dxdt = sigma * (y - x)
    dydt = x * (rho - z) - y
    dzdt = x * y - beta * z
    return [dxdt, dydt, dzdt]

# Solve Lorenz equations
sol = solve_ivp(lorenz, [0, 100], [0.1, 0.1, 0.1], t_eval=np.linspace(0, 100, 10000))

# Prepare data for plotting
x, y, z = sol.y

figure = dict(
    data=[{"x": [], "y": [], 'mode': 'markers', 'marker': {'color': 'red', 'size': 10}, 'name': 'Start'},
          {"x": [], "y": [], 'mode': 'markers', 'marker': {'color': 'blue', 'size': 10}, 'name': 'End'}],
    layout=dict(xaxis=dict(range=[-25, 25]), yaxis=dict(range=[-25, 25])),
)

app = Dash(__name__, update_title=None)
app.layout = html.Div(
    [
        html.H4("Lorenz System"),
        dcc.Graph(id="graph", figure=dict(figure)),
        dcc.Interval(id="interval", interval=25, max_intervals=400),
        dcc.Store(id="offset", data=0),
        dcc.Store(id="store", data=dict(x=x, y=y)),
    ]
)

clientside_callback(
    """
    function (n_intervals, data, offset) {
        offset = offset % data.x.length;
        const end = Math.min((offset + 10), data.x.length);
        return [[{x: [data.x.slice(offset, end)], y: [data.y.slice(offset, end)]}, [0], 500], end]
    }
    """,
    [
        Output("graph", "extendData"),
        Output("offset", "data"),
    ],
    [Input("interval", "n_intervals")],
    [State("store", "data"), State("offset", "data")],
)

@app.callback(
    Output("graph", "extendData"),
    [Input("interval", "n_intervals")],
    [State("store", "data"), State("offset", "data")]
)
def update_trace(n_intervals, data, offset):
    offset = offset % len(data['x'])
    end = min(offset + 10, len(data['x']))

    extend_data = {
        'x[0]': [[data['x'][0]]],
        'y[0]': [[data['y'][0]]],
        'x[1]': [[data['x'][-1]]],
        'y[1]': [[data['y'][-1]]]
    }

    return extend_data

if __name__ == "__main__":
    app.run_server()


In [4]:
pip install dash

Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy as np
from dash import Dash, html, dcc, Input, Output, clientside_callback
from scipy.integrate import solve_ivp

# Lorenz equations
def lorenz(t, state, sigma=10, rho=28, beta=8/3):
    x, y, z = state
    dxdt = sigma * (y - x)
    dydt = x * (rho - z) - y
    dzdt = x * y - beta * z
    return [dxdt, dydt, dzdt]

# Solve Lorenz equations
sol = solve_ivp(lorenz, [0, 100], [0.1, 0.1, 0.1], t_eval=np.linspace(0, 100, 10000))

# Prepare data for plotting
x, y, z = sol.y

figure = dict(
    data=[{"x": [], "y": []}],
    layout=dict(xaxis=dict(range=[-25, 25]), yaxis=dict(range=[-25, 25])),
)

app = Dash(__name__, update_title=None)
app.layout = html.Div(
    [
        html.H4("Lorenz System"),
        dcc.Graph(id="graph", figure=dict(figure)),
        dcc.Interval(id="interval", interval=25, max_intervals=400),
        dcc.Store(id="offset", data=0),
        dcc.Store(id="store", data=dict(x=x, y=y)),
    ]
)

clientside_callback(
    """
    function (n_intervals, data, offset) {
        offset = offset % data.x.length;
        const end = Math.min((offset + 10), data.x.length);
        return [[{x: [data.x.slice(offset, end)], y: [data.y.slice(offset, end)]}, [0], 500], end]
    }
    """,
    [
        Output("graph", "extendData"),
        Output("offset", "data"),
    ],
    [Input("interval", "n_intervals")],
    [State("store", "data"), State("offset", "data")],
)

if __name__ == "__main__":
    app.run_server()
